In [25]:

# TBD
from argparse import ArgumentParser, Namespace

parser = ArgumentParser(description="Compute MD5")
parser.add_argument("-c", action="extend", nargs="+", type=str, 
                        help="Compare two given partitions contents using MD5")
args = parser.parse_args()
print(args)

usage: ipykernel_launcher.py [-h] [-c C [C ...]]
ipykernel_launcher.py: error: unrecognized arguments: --f=c:\Users\acasa\AppData\Roaming\jupyter\runtime\kernel-v3d185641c2563206d411130cebc50f6c230cdbbb3.json


SystemExit: 2

C:\Users\acasa\AppData\Roaming\Python\Python313\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
from ast import literal_eval
from csv import DictReader, field_size_limit
from os import listdir, remove
from os.path import isdir, abspath, splitdrive
from sys import maxsize

field_size_limit(maxsize)

keyword = "processed"
in_path = ""
for entry in listdir(abspath('')):
    if not isdir(entry):
        if ".csv" in entry:
            if keyword in entry:
                remove(entry)
            else:
                in_path = entry
        
print(in_path)

sep = '.'
p,ext = in_path.split(sep)
duplicates_path = p + "-" + keyword + sep + ext
singles_path = p + "-" + keyword + "-" + "single" + sep + ext
print(singles_path, duplicates_path)

with open(duplicates_path, mode = "w", newline='', encoding="utf-8") as d_file:
    with open(singles_path, mode = "w", newline='', encoding="utf-8") as s_file:
        with open(in_path, mode = "r", newline='', encoding="utf-8") as in_file:
            reader = DictReader(in_file)
            for i, row in enumerate(reader):
                value = literal_eval(row["paths"])
                if "desktop.ini" not in value[0]: # ignore system files
                    # write duplicate paths once:
                    s = set()
                    for entry in value:
                        entry = splitdrive(entry)[1]
                        s.add(entry)

                    if len(s):
                        if len(s) == 1:
                            file = s_file
                        if len(value) > 2: # duplicates
                            file = d_file

                    file.write(f"\t{i}\t")
                    for entry in s:
                        file.write(f"{entry} \r\n")


output.csv
output-processed-single.csv output-processed.csv
